Librerías a importar

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np

from google.cloud import bigquery
from google.colab import auth
auth.authenticate_user()

from google.oauth2 import service_account

Mounted at /content/drive


Credenciales de google

In [ ]:
key_path = r'/content/drive/MyDrive/Dataton bot/Keys/dataton-bot-76300cc96f46.json'
project = 'dataton-bot'
credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)
client = bigquery.Client(credentials=credentials, project=credentials.project_id,)

01.01 Ingesta de datos - Lectura de df

In [ ]:
df = pd.read_csv('https://storage.googleapis.com/mef-peru/InversionPublica/DETALLE_INVERSIONES.csv')
df.columns = df.columns.str.replace('[Ñ]', 'N')

01.02 Ingesta de datos - Subida a Bigquery

In [ ]:
df.to_gbq('raw_data.detalle_inversiones',project_id=project,if_exists='replace')

1it [03:29, 209.76s/it]


02.01 Datos para tablero

In [ ]:
%%bigquery --project dataton-bot
CREATE or REPLACE TABLE `an_data.bi_inversiones` AS (


  SELECT 
  NIVEL, SECTOR, ENTIDAD, CODIGO_UNICO, CODIGO_SNIP, NOMBRE_INVERSION,
  ESTADO, SITUACION,
  MONTO_VIABLE, COSTO_ACTUALIZADO,
  FECHA_REGISTRO, FECHA_VIABILIDAD,
  FUNCION,
  MARCO, TIPO_INVERSION, DES_MODALIDAD,
  EXPEDIENTE_TECNICO, INFORME_CIERRE,
  DEVEN_ACUMUL_ANO_ANT, PIA_ANO_ACTUAL, PIM_ANO_ACTUAL,CERTIF_ANO_ACTUAL, COMPROM_ANUAL_ANO_ACTUAL, DEV_ANO_ACTUAL, SALDO_EJECUTAR,
  TIENE_F12B, TIENE_AVAN_FISICO, AVANCE_FISICO, AVANCE_EJECUCION,
  DEPARTAMENTO, PROVINCIA, DISTRITO, UBIGEO, LATITUD, LONGITUD, ANO_PROCESO,
  CONCAT(CAST(LATITUD AS STRING),",",CAST(LONGITUD AS STRING)) AS GEO_POINT,
 (COSTO_ACTUALIZADO-MONTO_VIABLE) AS DIFF_COSTO, 
  DATE_DIFF(CURRENT_DATE(),CAST(SUBSTR(FECHA_REGISTRO,1,10) AS DATE),DAY) AS DIFF_DATE,
  FROM raw_data.detalle_inversiones

);

""


02.02 Datos bot - indicadores y ranking

In [ ]:
%%bigquery --project dataton-bot
CREATE or REPLACE TABLE `an_data.ind_bot` AS (

SELECT * FROM
(
  SELECT 
  NIVEL, SECTOR, ENTIDAD, CODIGO_UNICO, CODIGO_SNIP, NOMBRE_INVERSION,
  DEPARTAMENTO, PROVINCIA, DISTRITO,
  AVANCE_EJECUCION, DIFF_COSTO, DIFF_DATE,
    RANK() OVER ( PARTITION BY DEPARTAMENTO ORDER BY -DIFF_DATE ) AS RANK_DATE_DIFF,
    RANK() OVER ( PARTITION BY DEPARTAMENTO ORDER BY -DIFF_COSTO ) AS RANK_COST_DIFF,
    RANK() OVER ( PARTITION BY DEPARTAMENTO ORDER BY AVANCE_EJECUCION ) AS RANK_AVANCE_EJEC
  FROM an_data.bi_inversiones
  WHERE AVANCE_EJECUCION IS NOT NULL AND AVANCE_EJECUCION > 0 AND DIFF_COSTO > 0 AND DEPARTAMENTO IS NOT NULL
)
WHERE RANK_DATE_DIFF<=10 OR RANK_COST_DIFF<=10 OR RANK_AVANCE_EJEC<=10

);

""
